In [7]:
# save as eia_tepc_onehour_test.py
# pip install requests?
import requests
from datetime import datetime, timedelta
import pprint

API_KEY = "cD1xkJzNn8RKvxYP8e9ZvIt8tru7mzBXHRdLAQZs"

BASE = "https://api.eia.gov/v2/electricity"
pp = pprint.PrettyPrinter(indent=2)

In [89]:
def get_json(url, params=None):
    params = params or {}
    params['api_key'] = API_KEY
    r = requests.get(url, params=params, timeout=30)
    r.raise_for_status()
    return r.json()

# list child routes under /v2/electricity
print("Fetching top-level electricity routes (discover child dataset IDs)...")
root = get_json(f"{BASE}")
# print route ids + names
routes = root.get("response", {}).get("routes", [])
print("Available child routes (id : name) — scanning for 'rto' / 'operational' entries:")
for rt in routes:
    print(f" - {rt.get('id')} : {rt.get('name')}")

# pick a likely child route: prefer 'rto' or 'electricity-power-operational-data'
candidate_ids = [r['id'] for r in routes if 'rto' in r['id'] or 'operational' in (r.get('name') or '').lower()]
if not candidate_ids:
    candidate_ids = [r['id'] for r in routes]  # fallback: show all and use first
print("\nCandidate route IDs:", candidate_ids)

route_id = candidate_ids[0]
print(f"\nQuerying metadata for route: {route_id}")
meta = get_json(f"{BASE}/{route_id}")
pp.pprint(meta.get("response", {}))

Fetching top-level electricity routes (discover child dataset IDs)...
Available child routes (id : name) — scanning for 'rto' / 'operational' entries:
 - retail-sales : Electricity Sales to Ultimate Customers
 - electric-power-operational-data : Electric Power Operations (Annual and Monthly)
 - rto : Electric Power Operations (Daily and Hourly)
 - state-electricity-profiles : State Specific Data
 - operating-generator-capacity : Inventory of Operable Generators
 - facility-fuel : Electric Power Operations for Individual Power Plants (Annual and Monthly)

Candidate route IDs: ['rto']

Querying metadata for route: rto
{ 'description': 'Hourly and daily electric power operations by balancing '
                 'authority.  \n'
                 '    Source: Form EIA-930',
  'id': 'rto',
  'name': 'Electric Power Operations (Daily and Hourly)',
  'routes': [ { 'description': 'Hourly demand, day-ahead demand forecast, net '
                               'generation, and interchange by balan

In [81]:
end_time = datetime.utcnow().replace(minute=0, second=0, microsecond=0)
start_time = end_time - timedelta(hours=1)

url = "https://api.eia.gov/v2/electricity/rto/region-data/data/"

params = {
    "api_key": API_KEY,
    "frequency": "hourly",
    "data[]": "value",
    "facets[respondent][]": "TEPC",
    "start": start_time.strftime("%Y-%m-%dT%H"),
    "end": end_time.strftime("%Y-%m-%dT%H"),
    "length": 10
}

resp = requests.get(url, params=params, timeout=30)
resp.raise_for_status()
pp.pprint(resp.json())

{ 'ExcelAddInVersion': '2.1.0',
  'apiVersion': '2.1.8',
  'request': { 'command': '/v2/electricity/rto/region-data/data/',
               'params': { 'api_key': 'cD1xkJzNn8RKvxYP8e9ZvIt8tru7mzBXHRdLAQZs',
                           'data': ['value'],
                           'end': '2025-09-30T22',
                           'facets': {'respondent': ['TEPC']},
                           'frequency': 'hourly',
                           'length': '10',
                           'start': '2025-09-30T21'}},
  'response': { 'data': [ { 'period': '2025-09-30T00',
                            'respondent': 'TEPC',
                            'respondent-name': 'Tucson Electric Power',
                            'type': 'D',
                            'type-name': 'Demand',
                            'value': '2166',
                            'value-units': 'megawatthours'},
                          { 'period': '2025-09-30T00',
                            'respondent': 'TEPC',
      

In [85]:
data = resp.json()
records = data.get("response", {}).get("data", [])
df = pd.DataFrame(records)
df.head()

,period,respondent,respondent-name,type,type-name,value,value-units
0,2025-09-30T00,TEPC,Tucson Electric Power,D,Demand,2166,megawatthours
1,2025-09-30T00,TEPC,Tucson Electric Power,DF,Day-ahead demand forecast,2151,megawatthours
2,2025-09-30T00,TEPC,Tucson Electric Power,NG,Net generation,2151,megawatthours
3,2025-09-30T00,TEPC,Tucson Electric Power,TI,Total interchange,-15,megawatthours


In [87]:
if not df.empty:
    df["period"] = pd.to_datetime(df["period"], utc=True)
    df = df.set_index("period")
df.head()

,respondent,respondent-name,type,type-name,value,value-units
period,,,,,,
2025-09-30 00:00:00+00:00,TEPC,Tucson Electric Power,D,Demand,2166,megawatthours
2025-09-30 00:00:00+00:00,TEPC,Tucson Electric Power,DF,Day-ahead demand forecast,2151,megawatthours
2025-09-30 00:00:00+00:00,TEPC,Tucson Electric Power,NG,Net generation,2151,megawatthours
2025-09-30 00:00:00+00:00,TEPC,Tucson Electric Power,TI,Total interchange,-15,megawatthours
